In [1]:
# Import the Z3 constraint module
from z3_react_constraints import (
    # Enums and constants
    AccessLevel,
    MIN_LLM_TOKENS, MIN_ITERATION_COST, MIN_ITERATION_TIME, SATISFACTION_THRESHOLD,
    
    # Runtime dataclasses
    AgentStateRT, ToolSpecRT,
    
    # Constraint checker
    ReactConstraintChecker,
    
    # Optimization
    optimize_tool_selection,
    
    # State update functions
    deduct_tool_cost, increment_iteration, update_satisfaction, mark_done,
    
    # Example tools
    EXAMPLE_TOOLS
)

print("Z3 constraint module loaded successfully!")
print(f"Satisfaction threshold: {SATISFACTION_THRESHOLD}%")
print(f"Available tools: {list(EXAMPLE_TOOLS.keys())}")

Z3 constraint module loaded successfully!
Satisfaction threshold: 90%
Available tools: ['read_file', 'write_file', 'run_python', 'web_search']


## Part 1: Basic Constraint Checking

Let's verify that Z3 correctly enforces permission constraints.

In [2]:
# Create a constraint checker
checker = ReactConstraintChecker()

# Create an agent state with READ access but NO execute permission
state = AgentStateRT(
    iteration=0,
    max_iterations=10,
    token_budget=5000,
    cost_budget=100,
    time_budget=30000,
    file_access=AccessLevel.READ,
    exec_allowed=False,
    satisfaction=0,
    done=False
)

print("Agent State:")
print(f"  File access: {state.file_access.name}")
print(f"  Execute allowed: {state.exec_allowed}")
print(f"  Budgets: tokens={state.token_budget}, cost={state.cost_budget}, time={state.time_budget}ms")

Agent State:
  File access: READ
  Execute allowed: False
  Budgets: tokens=5000, cost=100, time=30000ms


In [3]:
# Test each tool against the current state
print("Tool Permission Checks:")
print("=" * 60)

for name, tool in EXAMPLE_TOOLS.items():
    allowed, reason = checker.check_tool_invocation(tool, state)
    status = "✅ ALLOWED" if allowed else "❌ BLOCKED"
    print(f"\n{name}:")
    print(f"  Required: access={AccessLevel(tool.required_access).name}, exec={tool.requires_exec}")
    print(f"  Result: {status}")
    if reason:
        print(f"  Reason: {reason}")

Tool Permission Checks:

read_file:
  Required: access=READ, exec=False
  Result: ✅ ALLOWED

write_file:
  Required: access=READ_WRITE, exec=False
  Result: ❌ BLOCKED
  Reason: insufficient file access: need READ_WRITE, have READ

run_python:
  Required: access=NONE, exec=True
  Result: ❌ BLOCKED
  Reason: execute permission required but not granted

web_search:
  Required: access=NONE, exec=False
  Result: ✅ ALLOWED


### Expected Results

| Tool | Access Needed | Exec Needed | State Has | Result |
|------|---------------|-------------|-----------|--------|
| read_file | READ | No | READ, no exec | ✅ ALLOWED |
| write_file | READ_WRITE | No | READ, no exec | ❌ BLOCKED (insufficient access) |
| run_python | NONE | Yes | READ, no exec | ❌ BLOCKED (no execute) |
| web_search | NONE | No | READ, no exec | ✅ ALLOWED |

## Part 2: Continue vs Respond Decision

The agent must decide whether to continue (another iteration) or respond (final answer).

In [4]:
# Test different states for continue/respond decision

test_states = [
    ("Fresh state (low satisfaction)", AgentStateRT(
        iteration=0, max_iterations=10,
        token_budget=5000, cost_budget=100, time_budget=30000,
        file_access=AccessLevel.READ, exec_allowed=False,
        satisfaction=20, done=False
    )),
    ("High satisfaction (above threshold)", AgentStateRT(
        iteration=2, max_iterations=10,
        token_budget=5000, cost_budget=100, time_budget=30000,
        file_access=AccessLevel.READ, exec_allowed=False,
        satisfaction=95, done=False  # Above SATISFACTION_THRESHOLD (90)
    )),
    ("Max iterations reached", AgentStateRT(
        iteration=10, max_iterations=10,
        token_budget=5000, cost_budget=100, time_budget=30000,
        file_access=AccessLevel.READ, exec_allowed=False,
        satisfaction=50, done=False
    )),
    ("Token budget exhausted", AgentStateRT(
        iteration=3, max_iterations=10,
        token_budget=50, cost_budget=100, time_budget=30000,  # Below MIN_LLM_TOKENS (100)
        file_access=AccessLevel.READ, exec_allowed=False,
        satisfaction=40, done=False
    )),
    ("Marked as done", AgentStateRT(
        iteration=5, max_iterations=10,
        token_budget=5000, cost_budget=100, time_budget=30000,
        file_access=AccessLevel.READ, exec_allowed=False,
        satisfaction=80, done=True
    )),
]

print("Continue/Respond Decision Tests:")
print("=" * 60)

for name, test_state in test_states:
    decision = checker.check_continue_decision(test_state)
    emoji = {"continue": "🔄", "respond": "📤", "satisfied": "✨"}[decision]
    print(f"\n{name}:")
    print(f"  iter={test_state.iteration}/{test_state.max_iterations}, "
          f"sat={test_state.satisfaction}%, done={test_state.done}")
    print(f"  Decision: {emoji} {decision.upper()}")

Continue/Respond Decision Tests:

Fresh state (low satisfaction):
  iter=0/10, sat=20%, done=False
  Decision: 🔄 CONTINUE

High satisfaction (above threshold):
  iter=2/10, sat=95%, done=False
  Decision: ✨ SATISFIED

Max iterations reached:
  iter=10/10, sat=50%, done=False
  Decision: 📤 RESPOND

Token budget exhausted:
  iter=3/10, sat=40%, done=False
  Decision: 📤 RESPOND

Marked as done:
  iter=5/10, sat=80%, done=True
  Decision: 📤 RESPOND


## Part 3: Z3 Optimization for Tool Selection

Use Z3's `Optimize` to find the best permitted tool given constraints.

In [5]:
# Create a state with full permissions
full_access_state = AgentStateRT(
    iteration=0, max_iterations=10,
    token_budget=10000, cost_budget=500, time_budget=60000,
    file_access=AccessLevel.READ_WRITE,
    exec_allowed=True,
    satisfaction=0, done=False
)

tools_list = list(EXAMPLE_TOOLS.values())

print("Tool Costs:")
for tool in tools_list:
    print(f"  {tool.name}: cost={tool.base_cost}, time={tool.time_estimate}ms, tokens={tool.token_estimate}")

print("\nOptimization Results (full permissions):")
print("=" * 60)

for objective in ['min_cost', 'min_time', 'min_tokens']:
    best = optimize_tool_selection(tools_list, full_access_state, objective)
    print(f"  {objective}: {best.name if best else 'None'}")

Tool Costs:
  read_file: cost=1, time=100ms, tokens=500
  write_file: cost=2, time=200ms, tokens=300
  run_python: cost=5, time=5000ms, tokens=1000
  web_search: cost=10, time=2000ms, tokens=2000

Optimization Results (full permissions):
  min_cost: read_file
  min_time: read_file
  min_tokens: write_file


In [6]:
# Now test with limited permissions - some tools become unavailable
limited_state = AgentStateRT(
    iteration=0, max_iterations=10,
    token_budget=10000, cost_budget=500, time_budget=60000,
    file_access=AccessLevel.READ,  # Can't use write_file
    exec_allowed=False,            # Can't use run_python
    satisfaction=0, done=False
)

print("\nOptimization Results (limited permissions - READ only, no exec):")
print("=" * 60)

for objective in ['min_cost', 'min_time', 'min_tokens']:
    best = optimize_tool_selection(tools_list, limited_state, objective)
    print(f"  {objective}: {best.name if best else 'None'}")
    
print("\nNote: write_file and run_python are excluded due to permission constraints")


Optimization Results (limited permissions - READ only, no exec):
  min_cost: read_file
  min_time: read_file
  min_tokens: read_file

Note: write_file and run_python are excluded due to permission constraints


## Part 4: Simulating Agent Loop with Z3 Enforcement

Let's simulate a complete agent loop where Z3 enforces constraints at each step.

In [7]:
def simulate_agent_loop(initial_state: AgentStateRT, 
                        available_tools: list[ToolSpecRT],
                        max_steps: int = 20) -> list[dict]:
    """
    Simulate agent loop with Z3 constraint enforcement.
    
    At each step:
    1. Check if must_respond (Z3)
    2. If not, select optimal tool (Z3 Optimize)
    3. Verify tool is permitted (Z3)
    4. Execute tool and update state
    5. Simulate satisfaction increase
    6. Check continue/respond decision (Z3)
    """
    checker = ReactConstraintChecker()
    state = initial_state
    history = []
    
    for step in range(max_steps):
        step_info = {
            'step': step + 1,
            'iteration': state.iteration,
            'satisfaction': state.satisfaction,
            'token_budget': state.token_budget,
            'action': None,
            'tool': None,
            'decision': None
        }
        
        # Check if we must respond
        decision = checker.check_continue_decision(state)
        step_info['decision'] = decision
        
        if decision == 'respond':
            step_info['action'] = 'RESPOND (forced)'
            history.append(step_info)
            break
        elif decision == 'satisfied':
            step_info['action'] = 'RESPOND (satisfied)'
            history.append(step_info)
            break
        
        # Select optimal tool
        best_tool = optimize_tool_selection(available_tools, state, 'min_cost')
        
        if best_tool is None:
            step_info['action'] = 'RESPOND (no valid tools)'
            history.append(step_info)
            break
        
        # Verify tool invocation (double-check)
        allowed, reason = checker.check_tool_invocation(best_tool, state)
        if not allowed:
            step_info['action'] = f'ERROR: {reason}'
            history.append(step_info)
            break
        
        # Execute tool (simulated)
        step_info['action'] = 'INVOKE_TOOL'
        step_info['tool'] = best_tool.name
        
        # Update state
        state = deduct_tool_cost(state, best_tool)
        state = increment_iteration(state)
        
        # Simulate satisfaction increase (in real agent, this comes from LLM-as-judge)
        new_satisfaction = min(100, state.satisfaction + 15)  # +15% per iteration
        state = update_satisfaction(state, new_satisfaction)
        
        history.append(step_info)
    
    return history

# Run simulation
initial = AgentStateRT(
    iteration=0, max_iterations=10,
    token_budget=3000, cost_budget=50, time_budget=30000,
    file_access=AccessLevel.READ,
    exec_allowed=False,
    satisfaction=0, done=False
)

tools = [EXAMPLE_TOOLS['read_file'], EXAMPLE_TOOLS['web_search']]

print("Agent Loop Simulation")
print("=" * 70)
print(f"Initial: tokens={initial.token_budget}, cost={initial.cost_budget}, sat={initial.satisfaction}%")
print(f"Available tools: {[t.name for t in tools]}")
print()

history = simulate_agent_loop(initial, tools)

for h in history:
    print(f"Step {h['step']}: iter={h['iteration']}, sat={h['satisfaction']}%, "
          f"tokens={h['token_budget']}")
    print(f"  Action: {h['action']}" + (f" ({h['tool']})" if h['tool'] else ""))
    print(f"  Decision: {h['decision']}")
    print()

Agent Loop Simulation
Initial: tokens=3000, cost=50, sat=0%
Available tools: ['read_file', 'web_search']

Step 1: iter=0, sat=0%, tokens=3000
  Action: INVOKE_TOOL (read_file)
  Decision: continue

Step 2: iter=1, sat=15%, tokens=2500
  Action: INVOKE_TOOL (read_file)
  Decision: continue

Step 3: iter=2, sat=30%, tokens=2000
  Action: INVOKE_TOOL (read_file)
  Decision: continue

Step 4: iter=3, sat=45%, tokens=1500
  Action: INVOKE_TOOL (read_file)
  Decision: continue

Step 5: iter=4, sat=60%, tokens=1000
  Action: INVOKE_TOOL (read_file)
  Decision: continue

Step 6: iter=5, sat=75%, tokens=500
  Action: INVOKE_TOOL (read_file)
  Decision: continue

Step 7: iter=6, sat=90%, tokens=0
  Action: RESPOND (forced)
  Decision: respond



## Part 5: Formal Correspondence

The Python constraints directly mirror the ACL2 formal specification:

| ACL2 Function | Python Function | Description |
|---------------|-----------------|-------------|
| `access-sufficient-p` | `access_sufficient()` | File access level check |
| `tool-permitted-p` | `tool_permitted()` | Permission (access + exec) |
| `tool-budget-sufficient-p` | `tool_budget_sufficient()` | Budget check |
| `can-invoke-tool-p` | `can_invoke_tool()` | Combined permission + budget |
| `must-respond-p` | `must_respond()` | Forced response condition |
| `should-continue-p` | `should_continue()` | Continue condition |
| `deduct-tool-cost` | `deduct_tool_cost()` | State update after tool |
| `increment-iteration` | `increment_iteration()` | Iteration counter |
| `update-satisfaction` | `update_satisfaction()` | Satisfaction update |
| `mark-done` | `mark_done()` | Mark agent complete |

The ACL2 theorems proven in experiment-01 guarantee these properties hold:

1. **permission-safety**: `can-invoke-tool-p` implies `tool-permitted-p`
2. **tool-deduction-preserves-budget-nonneg**: Budgets stay non-negative
3. **iteration-increases**: Iteration strictly increases
4. **termination-by-iteration**: Max iterations forces `must-respond-p`

In [8]:
# Verify the ACL2-proven properties hold in our Z3 implementation
from z3 import Solver, Const, Not, Implies, ForAll, And, unsat
from z3_react_constraints import (
    AgentState, ToolSpec,
    can_invoke_tool, tool_permitted, must_respond
)

def verify_property(name: str, prop) -> bool:
    """Verify a property by checking if its negation is unsatisfiable."""
    s = Solver()
    s.add(Not(prop))
    result = s.check()
    passed = (result == unsat)
    print(f"{name}: {'✅ VERIFIED' if passed else '❌ FAILED'}")
    return passed

print("Verifying ACL2 Theorem Correspondence:")
print("=" * 60)

# Create symbolic variables
tool = Const('tool', ToolSpec)
state = Const('state', AgentState)

# Property 1: permission-safety
# can_invoke_tool => tool_permitted
prop1 = Implies(can_invoke_tool(tool, state), tool_permitted(tool, state))
verify_property("permission-safety (can_invoke => permitted)", prop1)

# Property 2: termination-by-iteration  
# iteration >= max_iterations => must_respond
prop2 = Implies(
    AgentState.iteration(state) >= AgentState.max_iterations(state),
    must_respond(state)
)
verify_property("termination-by-iteration", prop2)

Verifying ACL2 Theorem Correspondence:
permission-safety (can_invoke => permitted): ✅ VERIFIED
termination-by-iteration: ✅ VERIFIED


True

## Summary

Experiment 03 demonstrates:

1. **Z3 Datatypes** mirroring ACL2 FTY `defprod` types
2. **Constraint functions** matching ACL2 definitions exactly
3. **Runtime enforcement** using Z3 `Solver` to verify constraints before actions
4. **Optimization** using Z3 `Optimize` for tool selection
5. **Formal correspondence** with ACL2 proven properties

### Next Steps (Experiment 04)

- Integrate with LangGraph for actual LLM-powered agent
- Add LLM-as-judge for satisfaction scoring
- Connect to LM Studio for local model inference
- Full end-to-end verified agent execution